In [10]:


import os
import cv2
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers
from imblearn.over_sampling import SMOTE

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# ----------------------------
# Paths (YOUR paths)
# ----------------------------
train_csv_path = "/kaggle/input/diabetic-retinopathy-detection/trainLabels.csv.zip"
train_images_dir = "/kaggle/input/datasets/josephrynkiewicz/diabetic-retinopathy-train-unzipped/train"

# ----------------------------
# Load CSV
# ----------------------------
df = pd.read_csv(train_csv_path)

img_number = 10016
df = df.iloc[:img_number].copy()

# Binary label
df["Label"] = (df["level"] != 0).astype(int)

# Build filepath
df["filepath"] = df["image"].apply(lambda x: os.path.join(train_images_dir, f"{x}.jpeg"))

# Keep 10%
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

# ----------------------------
# Skip missing/corrupt files
# ----------------------------
df = df[df["filepath"].apply(os.path.exists)].reset_index(drop=True)

def is_readable(path):
    try:
        im = cv2.imread(path)
        return im is not None
    except Exception:
        return False

df = df[df["filepath"].apply(is_readable)].reset_index(drop=True)
print("Usable rows after filtering:", len(df))
print(df["Label"].value_counts())

# ----------------------------
# Split AFTER filtering
# ----------------------------
train_df, temp_df = train_test_split(df, test_size=0.4, stratify=df["Label"], random_state=42)
val_df, test_df   = train_test_split(temp_df, test_size=0.5, stratify=temp_df["Label"], random_state=42)

print("Train:", train_df.shape, "Val:", val_df.shape, "Test:", test_df.shape)


# ============================================================
# Step 2: Enhancements + Resize to 500x500
# ============================================================
IMG_SIZE = (500, 500)  # (H, W)

def enhance_none(img_rgb):
    return img_rgb

def enhance_clahe(img_rgb):
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l2 = clahe.apply(l)
    out = cv2.merge((l2, a, b))
    return cv2.cvtColor(out, cv2.COLOR_LAB2RGB)

def enhance_unsharp(img_rgb):
    blur = cv2.GaussianBlur(img_rgb, (0, 0), sigmaX=1.0)
    sharp = cv2.addWeighted(img_rgb, 1.5, blur, -0.5, 0)
    return sharp

def enhance_edges(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return cv2.addWeighted(img_rgb, 0.9, edges_rgb, 0.1, 0)

ENHANCERS = {
    "none": enhance_none,
    "clahe": enhance_clahe,
    "unsharp": enhance_unsharp,
    "edges": enhance_edges,
}

def load_image_cv2(filepath, enhancer_name="none"):
    """
    Safe loader. Returns float32 [0,1] image or None.
    NOTE: NO extra rescale later (avoid double normalization).
    """
    try:
        img = cv2.imread(filepath)
        if img is None:
            return None
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMG_SIZE, interpolation=cv2.INTER_AREA)
        img = ENHANCERS[enhancer_name](img)
        img = img.astype(np.float32) / 255.0
        return img
    except Exception:
        return None


# ============================================================
# Step 1 + Step 3 (fast): embeddings + SMOTE + classifier
# ============================================================
def build_feature_extractor(input_shape=(500, 500, 3)):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
        pooling="avg"
    )
    base.trainable = False
    return base

def compute_embeddings(df_subset, extractor, enhancer_name, batch_size=16):
    paths = df_subset["filepath"].tolist()
    labels = df_subset["Label"].values.astype(int)

    X_chunks, y_chunks = [], []

    for i in range(0, len(paths), batch_size):
        batch_paths = paths[i:i+batch_size]
        batch_y = labels[i:i+batch_size]

        imgs, ys = [], []
        for p, y in zip(batch_paths, batch_y):
            im = load_image_cv2(p, enhancer_name=enhancer_name)
            if im is not None:
                imgs.append(im)
                ys.append(y)

        if not imgs:
            continue

        arr = np.stack(imgs, axis=0)
        arr = tf.keras.applications.efficientnet.preprocess_input(arr * 255.0)

        emb = extractor.predict(arr, verbose=0)
        X_chunks.append(emb)
        y_chunks.append(np.array(ys, dtype=int))

    return np.concatenate(X_chunks, axis=0), np.concatenate(y_chunks, axis=0)

def train_smote_classifier(Xtr, ytr, Xva, yva, epochs=10):
    sm = SMOTE(random_state=42)
    Xb, yb = sm.fit_resample(Xtr, ytr)

    clf = tf.keras.Sequential([
        layers.Input(shape=(Xb.shape[1],)),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.4),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid")
    ])
    clf.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc"), "accuracy"]
    )
    clf.fit(Xb, yb, validation_data=(Xva, yva), epochs=epochs, batch_size=64, verbose=1)
    return clf

extractor = build_feature_extractor((IMG_SIZE[0], IMG_SIZE[1], 3))

results = {}
best_enh, best_auc = None, -1

for enh in ENHANCERS:
    print(f"\n=== Evaluating enhancement: {enh} ===")
    Xtr, ytr = compute_embeddings(train_df, extractor, enhancer_name=enh)
    Xva, yva = compute_embeddings(val_df, extractor, enhancer_name=enh)

    clf = train_smote_classifier(Xtr, ytr, Xva, yva, epochs=10)
    pred = clf.predict(Xva, verbose=0).ravel()
    auc = roc_auc_score(yva, pred)

    results[enh] = float(auc)
    print(f"Validation AUC ({enh}): {auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        best_enh = enh

print("\nEnhancement comparison (AUC):")
for k, v in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{k:>8}: {v:.4f}")
print(f"\nBest enhancement: {best_enh} (AUC={best_auc:.4f})")


# ============================================================
# End-to-end EfficientNet training (FIXED decode error)
# ============================================================
def make_tf_dataset(df_subset, enhancer_name, batch_size=8, training=False):
    paths = df_subset["filepath"].astype(str).values
    labels = df_subset["Label"].astype(np.float32).values

    def _py_load(path_tensor):
        # FIX: path_tensor is an EagerTensor -> convert to bytes -> decode
        p = path_tensor.numpy().decode("utf-8")
        img = load_image_cv2(p, enhancer_name=enhancer_name)
        if img is None:
            img = np.zeros((IMG_SIZE[0], IMG_SIZE[1], 3), dtype=np.float32)
        return img

    def _load(path, label):
        img = tf.py_function(_py_load, [path], Tout=tf.float32)
        img.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3])
        img = tf.keras.applications.efficientnet.preprocess_input(img * 255.0)
        return img, label

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.shuffle(512, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_tf_dataset(train_df, best_enh, batch_size=8, training=True)
val_ds   = make_tf_dataset(val_df, best_enh, batch_size=8, training=False)
test_ds  = make_tf_dataset(test_df, best_enh, batch_size=8, training=False)

# Class weights
neg = int((train_df["Label"] == 0).sum())
pos = int((train_df["Label"] == 1).sum())
class_weight = {0: (neg + pos) / (2.0 * neg), 1: (neg + pos) / (2.0 * pos)}
print("Class weight:", class_weight)

def build_efficientnet(input_shape=(500, 500, 3), base_trainable=False):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
        pooling="avg"
    )
    base.trainable = base_trainable

    inp = layers.Input(shape=input_shape)
    x = base(inp, training=False)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc"), "accuracy"]
    )
    return model

model = build_efficientnet((IMG_SIZE[0], IMG_SIZE[1], 3), base_trainable=False)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", patience=2, factor=0.5),
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=1
)

# Test
test_pred = model.predict(test_ds, verbose=0).ravel()
test_y = test_df["Label"].values.astype(int)

print("\nTEST AUC:", roc_auc_score(test_y, test_pred))
print("\nClassification report (threshold=0.5):")
print(classification_report(test_y, (test_pred >= 0.5).astype(int)))
print("Confusion matrix:\n", confusion_matrix(test_y, (test_pred >= 0.5).astype(int)))

model.save(f"efficientnet_best_{best_enh}_500x500.h5")
print(f"\nSaved: efficientnet_best_{best_enh}_500x500.h5")

Usable rows after filtering: 1002
Label
0    736
1    266
Name: count, dtype: int64
Train: (601, 4) Val: (200, 4) Test: (201, 4)

=== Evaluating enhancement: none ===
Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 211ms/step - accuracy: 0.4701 - auc: 0.5049 - loss: 0.7437 - val_accuracy: 0.7350 - val_auc: 0.5998 - val_loss: 0.6286
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5932 - auc: 0.6211 - loss: 0.6747 - val_accuracy: 0.4700 - val_auc: 0.5834 - val_loss: 0.7100
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6228 - auc: 0.6740 - loss: 0.6494 - val_accuracy: 0.6100 - val_auc: 0.6142 - val_loss: 0.6579
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6530 - auc: 0.7086 - loss: 0.6201 - val_accuracy: 0.7250 - val_auc: 0.6246 - val_loss: 0.5992
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6888 - auc: 0.7648 - loss: 0.5810 - val_accuracy: 0.5700 - val_auc: 0.6420 - val_loss: 0.6954
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 

2026-02-25 01:22:36.248420: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-25 01:22:36.386097: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-25 01:22:36.619438: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-25 01:22:36.755906: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-25 01:22:36.985435: E external/local_xla/xla/stream_

76/76 ━━━━━━━━━━━━━━━━━━━━ 79s 457ms/step - accuracy: 0.5916 - auc: 0.5809 - loss: 0.6751 - val_accuracy: 0.5750 - val_auc: 0.5235 - val_loss: 0.6805 - learning_rate: 1.0000e-04
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 38s 203ms/step - accuracy: 0.5349 - auc: 0.5319 - loss: 0.7235 - val_accuracy: 0.5850 - val_auc: 0.5335 - val_loss: 0.6816 - learning_rate: 1.0000e-04
Epoch 3/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 37s 197ms/step - accuracy: 0.5631 - auc: 0.5641 - loss: 0.6941 - val_accuracy: 0.5850 - val_auc: 0.5363 - val_loss: 0.6753 - learning_rate: 1.0000e-04
Epoch 4/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 37s 198ms/step - accuracy: 0.5650 - auc: 0.5730 - loss: 0.6869 - val_accuracy: 0.5900 - val_auc: 0.5497 - val_loss: 0.6765 - learning_rate: 1.0000e-04
Epoch 5/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 37s 198ms/step - accuracy: 0.5141 - auc: 0.5844 - loss: 0.6922 - val_accuracy: 0.5900 - val_auc: 0.5543 - val_loss: 0.6728 - learning_rate: 1.0000e-04
Epoch 6/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 37s 202ms/step - accuracy: 0


TEST AUC: 0.6306731259561449

Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0       0.79      0.69      0.74       148
           1       0.36      0.49      0.42        53

    accuracy                           0.64       201
   macro avg       0.58      0.59      0.58       201
weighted avg       0.68      0.64      0.65       201

Confusion matrix:
 [[102  46]
 [ 27  26]]

Saved: efficientnet_best_edges_500x500.h5
